# Temporal Convolutional Network (2017)

S. Bai, J. Z. Kolter, and V. Koltun, “An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling,” arXiv:1803.01271 [cs], Apr. 2018, Accessed: Apr. 14, 2022. [Online]. Available: http://arxiv.org/abs/1803.01271<br/>

TCN = 1D FCN + Causal Convolution<br/>

- Causal Convolution
- Dilation Convolution
- Residual Connection

![avatar](img/tcn.png)

In [60]:
from tensorflow.keras.layers import Conv1D, Dropout, Layer
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

from zipfile import ZipFile
import os

In [111]:
# * 探索代码
'''
每个batch      4个mini batch
每个mini batch 128个数据行
每个数据行      10个点
'''
input_shape = (4, 10, 128)
x = tf.random.normal(input_shape)
y = CausalConv1D(filters=32, kernel_size=3, dilation_rate=1)(x)
padding = tf.constant([[0, 0], [3-1, 0], [0, 0]])
tf.pad(y, padding).shape

TensorShape([4, 12, 32])

In [29]:
# 时间序列预测
master_url_root = "https://raw.githubusercontent.com/numenta/NAB/master/data/"

df_small_noise_url_suffix = "artificialNoAnomaly/art_daily_small_noise.csv"
df_small_noise_url = master_url_root + df_small_noise_url_suffix
df_small_noise = pd.read_csv(
    df_small_noise_url, parse_dates=True, index_col="timestamp"
)

In [95]:
values = df_small_noise.values
input_data = values[:-10]
targets = values[10:]
dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(
    input_data, targets, sequence_length=32)

In [96]:
for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (128, 32, 1)
Target shape: (128, 1)


In [108]:
inputs = keras.layers.Input(shape=(32,1))
tcn_out = TCN([1, 1, 1])(inputs)
outputs = keras.layers.Dense(1)(tcn_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.05), loss="mse")
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 32, 1)]           0         
                                                                 
 tcn_24 (TCN)                (None, 32, 1)             54        
                                                                 
 dense_6 (Dense)             (None, 32, 1)             2         
                                                                 
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(
    dataset_train,
    epochs=32,
    validation_data=dataset_train
)

Epoch 1/32
32/32 [==============================] - 2s 25ms/step - loss: 2285.9053 - val_loss: 680.6572
Epoch 2/32
32/32 [==============================] - 0s 12ms/step - loss: 715.6017 - val_loss: 240.7920
Epoch 3/32
32/32 [==============================] - 0s 12ms/step - loss: 281.3766 - val_loss: 176.5928
Epoch 4/32
32/32 [==============================] - 0s 12ms/step - loss: 203.1738 - val_loss: 159.5183
Epoch 5/32
32/32 [==============================] - 0s 12ms/step - loss: 178.2701 - val_loss: 155.2338
Epoch 6/32
32/32 [==============================] - 0s 12ms/step - loss: 169.7629 - val_loss: 154.5621
Epoch 7/32
32/32 [==============================] - 0s 12ms/step - loss: 164.4594 - val_loss: 154.0522
Epoch 8/32
32/32 [==============================] - 0s 12ms/step - loss: 162.3778 - val_loss: 153.9573
Epoch 9/32
32/32 [==============================] - 0s 12ms/step - loss: 161.1022 - val_loss: 153.9952
Epoch 10/32
32/32 [==============================] - 0s 11ms/step - loss

In [ ]:
# 定义参数
NUMBER_OF_COLUMNS = 2048

# Code Repo

因果卷积层

In [61]:
class CausalConv1D(Conv1D):
    def __init__(self, kernel_size, dilation_rate, filters):
        super(CausalConv1D, self).__init__(
            filters      = filters,
            kernel_size  = kernel_size,
            dilation_rate= dilation_rate
        )
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        inputs = tf.pad(inputs, tf.constant([(0, 0,), (1, 0), (0, 0)]) * padding)
        return super(CausalConv1D, self).call(inputs)

TCN块

In [62]:
class TemporalBlock(Layer):
    def __init__(self, kernel_size, dilation_rate, dropout=0.2, n_outputs=1):
        super(TemporalBlock, self).__init__()        
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = CausalConv1D(
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            filters=n_outputs)
        self.conv2 = CausalConv1D(
            kernel_size=kernel_size,
            dilation_rate=dilation_rate,
            filters=n_outputs)
        self.down_sample = None

    
    def build(self, input_shape):
        self.dropout1 = Dropout(self.dropout)
        self.dropout2 = Dropout(self.dropout)
        # channel_dim = 2
        # if input_shape[channel_dim] != self.n_outputs:
        #     self.down_sample = tf.layers.Dense(self.n_outputs, activation=None)
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = self.dropout2(x, training=training)
        # if self.down_sample is not None:
        #     inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)

TCN

In [107]:
class TCN(Layer):
    def __init__(self, num_channels, kernel_size=8, dropout=0.2):
        super(TCN, self).__init__()
        self.layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            out_channels = num_channels[i]
            self.layers.append(
                TemporalBlock(
                    n_outputs=out_channels, 
                    kernel_size=kernel_size, 
                    dilation_rate=dilation_size,
                    dropout=dropout)
            )
    
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs